In [ ]:
using PyCall

py"""
import pickle
 
def load_pickle(fpath):
    with open(fpath, "rb") as f:
        data = pickle.load(f)
    return data
"""

load_pickle = py"load_pickle"

Ws = load_pickle("/media/dan/Data/git/network_miner/julia_v_python_connectivity/python_outputs/Ws~epoch0.pkl")
# index like this: Ws[1,1] == python Ws[0][0]

3×14 Matrix{Vector{ComplexF64}}:
 [-0.00416117-4.86208e-18im, -0.0049611-0.00200327im, -0.00478543-0.00466064im, -0.00322968-0.00749588im, -0.000124787-0.00980932im, 0.00434712-0.0107961im, 0.00959711-0.0097219im, 0.0146615-0.00612467im, 0.0183328+1.21606e-17im, 0.0193803+0.00807926im  …  0.0168254-0.0169502im, 0.0193803-0.00807926im, 0.0183328-1.49643e-17im, 0.0146615+0.00612467im, 0.00959711+0.0097219im, 0.00434712+0.0107961im, -0.000124787+0.00980932im, -0.00322968+0.00749588im, -0.00478543+0.00466064im, -0.0049611+0.00200327im]  …  [-0.0245943-0.000252773im, 0.0648194-0.0133437im, -0.127572+0.052178im, 0.190925-0.128488im, -0.240404+0.239158im, 0.245764-0.369551im, -0.203345+0.488268im, 0.111362-0.565099im, -0.000993031+0.575659im, -0.104151-0.518863im, 0.168976+0.41009im, -0.189097-0.28177im, 0.16222+0.16296im, -0.114911-0.0763705im, 0.0610009+0.025426im]
 [-0.023527+0.00096174im, -0.0260059-0.00989803im, -0.0233636-0.0226138im, -0.0145658-0.0347149im, 0.000211955-0.0432279im, 0.0

In [4]:
using DSP
using FFTW
using Statistics
using LinearAlgebra
using Plots
using Printf
# using FilePathsBase


In [5]:
using DSP
using FFTW
using Statistics
using LinearAlgebra
# using Threads

function sinusoidal(a, f, sr, t, theta=0, DC=0)
    delta_i = 1 / sr
    f2pi = f * 2 * π
    nu = [DC + (a * sin(f2pi * i * delta_i + theta)) for i in 0:(t-1)]
    return nu
end

# Optimized zero pad function
function zero_pad!(data_padded, data)
    data_padded[1:length(data)] .= data
    data_padded[length(data)+1:end] .= 0
end

function coh(s_xx, s_yy, s_xy)
    # Compute the numerator: absolute value of the mean of s_xy along the last dimension
    con_num = abs.(mean(s_xy, dims=ndims(s_xy)))

    # Compute the denominator: square root of the product of means of s_xx and s_yy along the last dimension
    con_den = sqrt.(mean(s_xx, dims=ndims(s_xx)) .* mean(s_yy, dims=ndims(s_yy)))

    # Calculate coherence as the element-wise division of numerator by denominator
    coh = con_num ./ con_den
    return coh
end

function get_dpss_and_ratios(n, nw, ntapers; zerophase=false)
    # Compute DPSS windows
    windows = dpss(n, nw, ntapers; zerophase=zerophase)

    # Compute eigenvalues (concentration ratios)
    eigenvalues = dpsseig(windows, nw)

    return windows, eigenvalues
end

function next_fast_len(target::Int)::Int
    """
    Find the next fast size of input data to `fft`, for zero-padding, etc.

    Returns the next composite of the prime factors 2, 3, and 5 which is
    greater than or equal to `target`. (These are also known as 5-smooth
    numbers, regular numbers, or Hamming numbers.)

    Parameters
    ----------
    target : Int
        Length to start searching from. Must be a positive integer.

    Returns
    -------
    out : Int
        The first 5-smooth number greater than or equal to `target`.
    """
    # Precomputed Hamming numbers (5-smooth numbers) for quick lookup
    hams = [
        8, 9, 10, 12, 15, 16, 18, 20, 24, 25, 27, 30, 32, 36, 40, 45, 48, 50,
        54, 60, 64, 72, 75, 80, 81, 90, 96, 100, 108, 120, 125, 128, 135, 144,
        150, 160, 162, 180, 192, 200, 216, 225, 240, 243, 250, 256, 270, 288,
        300, 320, 324, 360, 375, 384, 400, 405, 432, 450, 480, 486, 500, 512,
        540, 576, 600, 625, 640, 648, 675, 720, 729, 750, 768, 800, 810, 864,
        900, 960, 972, 1000, 1024, 1080, 1125, 1152, 1200, 1215, 1250, 1280,
        1296, 1350, 1440, 1458, 1500, 1536, 1600, 1620, 1728, 1800, 1875, 1920,
        1944, 2000, 2025, 2048, 2160, 2187, 2250, 2304, 2400, 2430, 2500, 2560,
        2592, 2700, 2880, 2916, 3000, 3072, 3125, 3200, 3240, 3375, 3456, 3600,
        3645, 3750, 3840, 3888, 4000, 4050, 4096, 4320, 4374, 4500, 4608, 4800,
        4860, 5000, 5120, 5184, 5400, 5625, 5760, 5832, 6000, 6075, 6144, 6250,
        6400, 6480, 6561, 6750, 6912, 7200, 7290, 7500, 7680, 7776, 8000, 8100,
        8192, 8640, 8748, 9000, 9216, 9375, 9600, 9720, 10000
    ]

    if target <= 6
        return target
    end

    # Check if target is already a power of 2
    if (target & (target - 1)) == 0
        return target
    end

    # Quick lookup for small sizes
    if target <= hams[end]
        idx = searchsortedfirst(hams, target)
        return hams[idx]
    end

    # Function to compute the bit length of an integer
    bit_length(x::Int) = x <= 0 ? 0 : floor(Int, log2(x)) + 1

    match = typemax(Int)  # Initialize with maximum possible integer
    p5 = 1
    while p5 < target
        p35 = p5
        while p35 < target
            # Ceiling integer division
            quotient = cld(target, p35)
            p2 = 2^bit_length(quotient - 1)
            N = p2 * p35
            if N == target
                return N
            elseif N < match
                match = N
            end
            p35 *= 3
            if p35 == target
                return p35
            end
        end
        if p35 < match
            match = p35
        end
        p5 *= 5
        if p5 == target
            return p5
        end
    end
    if p5 < match
        match = p5
    end
    return match
end

function _get_nfft(Ws, X)::Int
    max_len = maximum([length(Wk) for Wk in Ws])
    n = last(size(X))
    nfft = n + max_len - 1
    # @show nfft
    nfft = next_fast_len(nfft)
    return nfft
end

# Precompute FFTs of Ws
function precompute_fft_Ws(Ws, nfft)
    n_taps, n_freqs = size(Ws)
    fft_Ws = Array{ComplexF64,3}(undef, n_taps, n_freqs, nfft)
    for taper_idx = 1:n_taps
        for freq_idx = 1:n_freqs
            W = Ws[taper_idx, freq_idx]
            padded_W = zeros(ComplexF64, nfft)
            zero_pad!(padded_W, W)
            fft_Ws[taper_idx, freq_idx, :] = fft(padded_W)
        end
    end
    return fft_Ws
end

# Precompute FFTs of X
function precompute_fft_X(X, nfft)
    n_epochs, n_channels, n_times = size(X)
    fft_X = Array{ComplexF64,3}(undef, n_epochs, n_channels, nfft)
    for epoch_idx = 1:n_epochs
        for channel_idx = 1:n_channels
            x = X[epoch_idx, channel_idx, :]
            padded_x = zeros(ComplexF64, nfft)
            zero_pad!(padded_x, x)
            fft_X[epoch_idx, channel_idx, :] = fft(padded_x)
        end
    end
    return fft_X
end


# Main function to compute tfr
function compute_tfr(X, Ws, nfft)
    n_epochs, n_channels, n_times = size(X)
    n_taps, n_freqs = size(Ws)
    tfr = zeros(ComplexF64, n_epochs, n_channels, n_taps, n_freqs, n_times)

    # Precompute FFTs
    fft_Ws = precompute_fft_Ws(Ws, nfft)
    fft_X = precompute_fft_X(X, nfft)

    # Loop over tapers and frequencies
    for taper_idx = 1:n_taps
        for freq_idx = 1:n_freqs
            fft_W = fft_Ws[taper_idx, freq_idx, :]
            W_size = length(Ws[taper_idx, freq_idx])
            total_size = n_times + W_size - 1
            ret_size = total_size
            # Preallocate ret array
            ret = zeros(ComplexF64, ret_size)

            for epoch_idx = 1:n_epochs
                for channel_idx = 1:n_channels
                    fx = fft_X[epoch_idx, channel_idx, :]
                    product = fx .* fft_W
                    ret .= ifft(product)[1:ret_size]

                    # Center the result
                    start = Int(floor((ret_size - n_times) / 2)) + 1
                    end_time = start + n_times - 1
                    tfr[epoch_idx, channel_idx, taper_idx, freq_idx, :] = ret[start:end_time]
                end
            end
        end
    end
    return tfr, fft_Ws, fft_X
end

compute_tfr (generic function with 1 method)

In [6]:
t = 32
sr = 32
f = 2
noise = 1000

v = sinusoidal(10, f, sr, t * 4, 0)
w = sinusoidal(10, f, sr, t * 4, π / 4)
y = sinusoidal(10, f, sr, t * 4, π / 2)
z = sinusoidal(10, f, sr, t * 4, π)

data = Array{Float64}(undef, 2, 4, 128)

data[1, :, :] = hcat(v, w, y, z)'
data[2, :, :] = hcat(-v, -w, -y, -z)';
size(data)

(2, 4, 128)

In [98]:
freqs = collect(2:15)  # inclusive of end
n_freqs = length(freqs)
mt_bandwidth = 4
n_taps = floor(Int, mt_bandwidth - 1)
n_cycles = 7
sfreq = 32
zero_mean = true
X = data;

In [ ]:
pre

In [99]:
tfr, fft_Ws, fft_X = compute_tfr(X, Ws, nfft);

In [100]:
using NPZ

# Load the .npy file
python_tfr = npzread("/media/dan/Data/git/network_miner/tfr_python.npy");

In [101]:
size(python_tfr)

(1, 4, 3, 14, 128)

In [103]:
size(tfr)

(2, 4, 3, 14, 128)